In [2]:
from numpy.random import seed
seed(7532)
from tensorflow import set_random_seed
set_random_seed(7532)

import numpy as np
import pandas as pd

from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.layers import BatchNormalization, Dense, Dropout, Conv2D, Flatten, MaxPool2D

Read in the data.

In [3]:
train_set = pd.read_csv('train_set_metadata.csv')
valid_set = pd.read_csv('valid_set_metadata.csv')
test_set = pd.read_csv('test_set_metadata.csv')

train_set_partition = np.load('train_set_partition.npy')

Compute the number of parts the train set was partitioned into.

In [4]:
n_parts = len(train_set_partition) - 1
n_parts

20

## CNN Model Training from Scratch (No Data Augmentation) ##

In [5]:
BEST_MODEL_PATH = 'best_model.hdf5'
INPUT_SHAPE = (320, 320, 3)

LEARNING_RATE = 0.0001
N_EPOCHS = 50
BATCH_SIZE = 32

In [6]:
def create_sequential_model(input_shape):
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=3, activation='relu',padding='same', 
                     input_shape=input_shape))
    model.add(MaxPool2D(2))
    model.add(BatchNormalization())
    
    model.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPool2D(4))
    model.add(BatchNormalization())

    model.add(Conv2D(128, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPool2D(8))
    model.add(BatchNormalization())  
    
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(512))
    model.add(Dropout(0.5))
    model.add(Dense(512))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    return model

In [7]:
model = create_sequential_model(INPUT_SHAPE)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 320, 320, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 160, 160, 32)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 160, 160, 32)      128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 160, 160, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 40, 40, 64)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 40, 40, 64)        256       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 40, 40, 128)       73856     
__________

In [7]:
gender_column_position = train_set.columns.get_loc('gender')

X_valid = np.load('valid_set_hmgd_arr.npy')
y_valid = valid_set['gender'].values

for part in range(n_parts):        
    print(f'Part {part + 1}:')
    
    train_filename = 'train_set_hmgd_arr_' + str(part + 1).zfill(2) + '.npy'
    subrange = range(train_set_partition[part], train_set_partition[part + 1])    
    X_train = np.load(train_filename)
    y_train = train_set.iloc[subrange, gender_column_position].values
    
    # Model initialization/load
    if part:
        model = load_model(BEST_MODEL_PATH)
    else:
        model = create_sequential_model(X_train.shape[1:])
        adam = Adam(lr=LEARNING_RATE)
        model.compile(optimizer=adam, loss='binary_crossentropy', 
                      metrics=['binary_accuracy'])
        
        # Initialize callbacks
        checkpoint = ModelCheckpoint(BEST_MODEL_PATH, monitor='val_loss', 
                                     save_best_only=True, save_weights_only=False)
        lr_reduction = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=6)
        early_stopping = EarlyStopping(monitor='val_loss', patience=12)    
        callback_list = [checkpoint, lr_reduction, early_stopping]
    
    model.fit(X_train, y_train, epochs=N_EPOCHS, batch_size=BATCH_SIZE, 
              callbacks=callback_list, validation_data=(X_valid, y_valid))
    
    #free up memory
    del X_train

Part 1:
Train on 8271 samples, validate on 4176 samples
Epoch 1/50
8271/8271 [==============================] - 84s 10ms/step - loss: 1.3773 - binary_accuracy: 0.6610 - val_loss: 0.6208 - val_binary_accuracy: 0.7397
Epoch 2/50
8271/8271 [==============================] - 64s 8ms/step - loss: 1.1494 - binary_accuracy: 0.6780 - val_loss: 0.6084 - val_binary_accuracy: 0.7634
Epoch 3/50
8271/8271 [==============================] - 64s 8ms/step - loss: 1.0002 - binary_accuracy: 0.7100 - val_loss: 0.5894 - val_binary_accuracy: 0.7677
Epoch 4/50
8271/8271 [==============================] - 63s 8ms/step - loss: 0.9373 - binary_accuracy: 0.7179 - val_loss: 0.5982 - val_binary_accuracy: 0.7756
Epoch 5/50
8271/8271 [==============================] - 64s 8ms/step - loss: 0.8734 - binary_accuracy: 0.7304 - val_loss: 0.5969 - val_binary_accuracy: 0.7866
Epoch 6/50
8271/8271 [==============================] - 64s 8ms/step - loss: 0.8226 - binary_accuracy: 0.7443 - val_loss: 0.5539 - val_binary_accura

8271/8271 [==============================] - 65s 8ms/step - loss: 0.2200 - binary_accuracy: 0.9091 - val_loss: 0.3685 - val_binary_accuracy: 0.8570
Epoch 16/50
8271/8271 [==============================] - 65s 8ms/step - loss: 0.2067 - binary_accuracy: 0.9144 - val_loss: 0.3728 - val_binary_accuracy: 0.8551
Epoch 17/50
8271/8271 [==============================] - 65s 8ms/step - loss: 0.2051 - binary_accuracy: 0.9139 - val_loss: 0.3742 - val_binary_accuracy: 0.8539
Epoch 18/50
8271/8271 [==============================] - 65s 8ms/step - loss: 0.2029 - binary_accuracy: 0.9196 - val_loss: 0.3777 - val_binary_accuracy: 0.8515
Part 5:
Train on 8271 samples, validate on 4176 samples
Epoch 1/50
8271/8271 [==============================] - 69s 8ms/step - loss: 0.3417 - binary_accuracy: 0.8542 - val_loss: 0.3528 - val_binary_accuracy: 0.8525
Epoch 2/50
8271/8271 [==============================] - 65s 8ms/step - loss: 0.3198 - binary_accuracy: 0.8653 - val_loss: 0.3552 - val_binary_accuracy: 0.850

8271/8271 [==============================] - 65s 8ms/step - loss: 0.3230 - binary_accuracy: 0.8672 - val_loss: 0.3645 - val_binary_accuracy: 0.8376
Epoch 4/50
8271/8271 [==============================] - 65s 8ms/step - loss: 0.3132 - binary_accuracy: 0.8706 - val_loss: 0.3495 - val_binary_accuracy: 0.8499
Epoch 5/50
8271/8271 [==============================] - 65s 8ms/step - loss: 0.3027 - binary_accuracy: 0.8756 - val_loss: 0.3722 - val_binary_accuracy: 0.8338
Epoch 6/50
8271/8271 [==============================] - 65s 8ms/step - loss: 0.2958 - binary_accuracy: 0.8795 - val_loss: 0.3413 - val_binary_accuracy: 0.8554
Epoch 7/50
8271/8271 [==============================] - 65s 8ms/step - loss: 0.2813 - binary_accuracy: 0.8854 - val_loss: 0.3613 - val_binary_accuracy: 0.8455
Epoch 8/50
8271/8271 [==============================] - 65s 8ms/step - loss: 0.2819 - binary_accuracy: 0.8831 - val_loss: 0.3569 - val_binary_accuracy: 0.8551
Epoch 9/50
8271/8271 [==============================] - 6

8271/8271 [==============================] - 64s 8ms/step - loss: 0.2452 - binary_accuracy: 0.8982 - val_loss: 0.3619 - val_binary_accuracy: 0.8470
Epoch 12/50
8271/8271 [==============================] - 64s 8ms/step - loss: 0.2223 - binary_accuracy: 0.9115 - val_loss: 0.3548 - val_binary_accuracy: 0.8534
Epoch 13/50
8271/8271 [==============================] - 65s 8ms/step - loss: 0.2113 - binary_accuracy: 0.9179 - val_loss: 0.3629 - val_binary_accuracy: 0.8520
Epoch 14/50
8271/8271 [==============================] - 65s 8ms/step - loss: 0.2061 - binary_accuracy: 0.9173 - val_loss: 0.3603 - val_binary_accuracy: 0.8534
Epoch 15/50
8271/8271 [==============================] - 65s 8ms/step - loss: 0.2032 - binary_accuracy: 0.9154 - val_loss: 0.3645 - val_binary_accuracy: 0.8520
Epoch 16/50
8271/8271 [==============================] - 65s 8ms/step - loss: 0.1980 - binary_accuracy: 0.9180 - val_loss: 0.3638 - val_binary_accuracy: 0.8527
Epoch 17/50
8271/8271 [=============================

Train on 8271 samples, validate on 4176 samples
Epoch 1/50
8271/8271 [==============================] - 69s 8ms/step - loss: 0.3286 - binary_accuracy: 0.8653 - val_loss: 0.3315 - val_binary_accuracy: 0.8602
Epoch 2/50
8271/8271 [==============================] - 65s 8ms/step - loss: 0.3046 - binary_accuracy: 0.8738 - val_loss: 0.3429 - val_binary_accuracy: 0.8575
Epoch 3/50
8271/8271 [==============================] - 65s 8ms/step - loss: 0.2954 - binary_accuracy: 0.8809 - val_loss: 0.3338 - val_binary_accuracy: 0.8635
Epoch 4/50
8271/8271 [==============================] - 65s 8ms/step - loss: 0.2883 - binary_accuracy: 0.8847 - val_loss: 0.3321 - val_binary_accuracy: 0.8649
Epoch 5/50
8271/8271 [==============================] - 65s 8ms/step - loss: 0.2723 - binary_accuracy: 0.8879 - val_loss: 0.3349 - val_binary_accuracy: 0.8621
Epoch 6/50
8271/8271 [==============================] - 66s 8ms/step - loss: 0.2633 - binary_accuracy: 0.8935 - val_loss: 0.3430 - val_binary_accuracy: 0.859

Read in the test data and evaluate the model.

In [8]:
X_test = np.load('test_set_hmgd_arr.npy')

In [9]:
model.evaluate(X_test, test_set['gender'].values)

7621/7621 [==============================] - 25s 3ms/step


[0.3699162668161013, 0.8518567117176223]

In [10]:
# free up memory
del X_test